<a href="https://colab.research.google.com/github/smartinternz02/SI-GuidedProject-610134-1699032511/blob/main/CrimeVision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
#upload kaggle.json to access Kaggle Datasets
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
#download ucf-crime-dataset
!kaggle datasets download -d odins0n/ucf-crime-dataset

100% 11.0G/11.0G [01:48<00:00, 169MB/s]
100% 11.0G/11.0G [01:48<00:00, 109MB/s]


In [5]:
!unzip ucf-crime-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: Train/Vandalism/Vandalism035_x264_230.png  
  inflating: Train/Vandalism/Vandalism035_x264_240.png  
  inflating: Train/Vandalism/Vandalism035_x264_250.png  
  inflating: Train/Vandalism/Vandalism035_x264_260.png  
  inflating: Train/Vandalism/Vandalism035_x264_270.png  
  inflating: Train/Vandalism/Vandalism035_x264_280.png  
  inflating: Train/Vandalism/Vandalism035_x264_290.png  
  inflating: Train/Vandalism/Vandalism035_x264_30.png  
  inflating: Train/Vandalism/Vandalism035_x264_300.png  
  inflating: Train/Vandalism/Vandalism035_x264_310.png  
  inflating: Train/Vandalism/Vandalism035_x264_320.png  
  inflating: Train/Vandalism/Vandalism035_x264_330.png  
  inflating: Train/Vandalism/Vandalism035_x264_340.png  
  inflating: Train/Vandalism/Vandalism035_x264_350.png  
  inflating: Train/Vandalism/Vandalism035_x264_360.png  
  inflating: Train/Vandalism/Vandalism035_x264_370.png  
  inflating: Train/Vandalism/Vandalism

In [6]:
#import necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, AveragePooling2D, Dropout,MaxPooling2D, Conv2D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import preprocess_input
from keras.applications import ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image

In [7]:
train_path = '/content/Train'
test_path = '/content/Test'

In [8]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [9]:
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [10]:
train_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1266345 images belonging to 14 classes.


In [11]:
test_set = test_datagen.flow_from_directory(test_path,
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 111308 images belonging to 14 classes.


In [12]:
class_names = list(train_set.class_indices.keys())
class_names = np.array(class_names)
class_names

array(['Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Explosion',
       'Fighting', 'NormalVideos', 'RoadAccidents', 'Robbery', 'Shooting',
       'Shoplifting', 'Stealing', 'Vandalism'], dtype='<U13')

##Model Building

In [13]:
#Using CNN and Transfer learning (pretrained model used - ResNet50)
model = Sequential()
resnet50 = ResNet50(include_top=False,
                   input_shape=(64,64,3),
                   pooling='avg',classes=14,
                   weights='imagenet')

for layer in resnet50.layers:
        layer.trainable=False

model.add(resnet50)
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation="relu"))
model.add(Dense(14, activation='softmax'))
model.summary()

94765736/94765736 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 module_wrapper (ModuleWrap  (None, 2048)              0         
 per)                                                            
                                                                 
 module_wrapper_1 (ModuleWr  (None, 256)               524544    
 apper)                                                          
                                                                 
 module_wrapper_2 (ModuleWr  (None, 256)               0         
 apper)                                                          
                                                                 
 module_wrapper_3 (ModuleWr  (None, 512)               13

In [14]:
#Compile model (other optimizers may result in better accuracy)
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

##Training the Model

In [18]:
#Checkpoint used to save the best performances only
#EarlyStopping used to reduce training time
checkpoint = ModelCheckpoint(
    filepath='crime.h5',
    monitor="val_loss",
    mode="min",
    save_best_only=True,
    save_weights_only=True,
    verbose=1)

# early_stopping_monitor = EarlyStopping(
#     monitor="val_loss",
#     min_delta=0,
#     patience=0,
#     mode="auto",
#     restore_best_weights=True,
#     verbose=1,
# )

In [20]:
#training the model (taking a lotttt of time, but it works)
callbacks = [checkpoint]
epochs=10

#zyada hi time mile toh increase epochs for training, you might also want to increase steps_per_epoch idk..
model_history = model.fit(
  train_set,
  validation_data=test_set,
  epochs=epochs,
  steps_per_epoch = 100,
  callbacks=callbacks
)

Epoch 1/10
100/100 [==============================] - ETA: 0s - loss: 0.8558 - accuracy: 0.7834
Epoch 1: val_loss improved from 1.92810 to 1.88830, saving model to crime.h5
100/100 [==============================] - 279s 3s/step - loss: 0.8558 - accuracy: 0.7834 - val_loss: 1.8883 - val_accuracy: 0.5832
Epoch 2/10
100/100 [==============================] - ETA: 0s - loss: 0.8457 - accuracy: 0.7894
Epoch 2: val_loss improved from 1.88830 to 1.82678, saving model to crime.h5
100/100 [==============================] - 275s 3s/step - loss: 0.8457 - accuracy: 0.7894 - val_loss: 1.8268 - val_accuracy: 0.5426
Epoch 3/10
100/100 [==============================] - ETA: 0s - loss: 0.8228 - accuracy: 0.7922
Epoch 3: val_loss did not improve from 1.82678
100/100 [==============================] - 276s 3s/step - loss: 0.8228 - accuracy: 0.7922 - val_loss: 1.8370 - val_accuracy: 0.5751
Epoch 4/10
100/100 [==============================] - ETA: 0s - loss: 0.7857 - accuracy: 0.8016
Epoch 4: val_loss d

##Testing the Model

In [23]:
from keras.models import load_model
model_predict = Sequential()
resnet50 = ResNet50(include_top=False,
                   input_shape=(64,64,3),
                   pooling='avg',classes=14,
                   weights='imagenet')

for layer in resnet50.layers:
        layer.trainable=False

model_predict.add(resnet50)
model_predict.add(Flatten())
model_predict.add(Dense(256, activation="relu"))
model_predict.add(Dropout(0.2))
model_predict.add(Dense(512, activation='relu'))
model_predict.add(Dropout(0.2))
model_predict.add(Dense(1024, activation="relu"))
model_predict.add(Dense(14, activation='softmax'))
model_predict.load_weights('crime.h5')

In [29]:
#fully trained model hopefully predicts better than it does rn :)
from keras.utils import load_img
img = image.load_img('/content/Test/NormalVideos/Normal_Videos_003_x264_2200.png',target_size=(64,64))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
prediction = np.argmax(model_predict.predict(x))
class_names[prediction]

1/1 [==============================] - 0s 39ms/step


'NormalVideos'